In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


In [2]:
import pickle
img_caption_file_name = '/content/drive/MyDrive/mscoco_40k_clustered.pkl'

with open(img_caption_file_name, 'rb') as f:
  df_mscoco = pickle.load(f)

The openai secret key can be created by setting up an account on https://platform.openai.com/docs/overview, then load in a minimum credit of $5, and creating a project api key on https://platform.openai.com/api-keys. Information on rate limits and costs are here https://platform.openai.com/docs/guides/rate-limits and https://openai.com/api/pricing/.

For a batch size of 25, it is recommended to load \$50 to enter tier 1 user status. For a bartch size of 35, it is recommended to load \$100 to enter tier 2 user status.

One run of 500 queries rakes roughly 1000 seconds and costs \$25.

Decomposing

In [6]:
import nest_asyncio
import asyncio
import aiohttp
import ssl
import certifi
import time
import pandas as pd
import pickle

nest_asyncio.apply()

# Settings
MODEL = "gpt-4o"
OPENAI_SECRET_KEY = "sk-proj-X...X"
MAX_RETRIES = 3  # Maximum number of retries
BATCH_SIZE = 25

# Asynchronous function to decompose a sentence
async def decompose_sentence_async(session, sentence, retries=MAX_RETRIES):
    payload = {
        'model': MODEL,
        'messages': [
            {"role": "user", "content": f"Example 1:\nOriginal: An institutional looking room holds an exit sign, chairs, recording equipment and a man with one arm aloft that appears to be demonstrating something with a device in the held-up hand for the benefit of a woman behind him that is watching closely.\n\nDecomposed form: \"An exit sign.\"|\"Chairs.\"|\"Recording equipment.\"|\"A man with one arm aloft appears to be demonstrating with a device in the held-up hand.\"|\"A woman behind a man is watching him closely.\"\n\nExample 2:\nOriginal: Black night, a blurry shot shows a tall fence that angles, in a kind of Z shape, around what appears to be a parking area with a large building fronting it, as a kid with a red helmet takes a fierce swing in the batting area, inside the fence.\n\nDecomposed form: \"In a black night, a blurry shot shows a tall fence that angles, in a kind of Z shape.\"|\"Around appears to be a parking area with a large building fronting it.\"|\"A kid with a red helmet takes a fierce swing in the batting area, inside the fence.\"\n\nExample 3:\nOriginal: An older looking model computer components; two disk drive boxes stacked, a keyboard unit, and a more modern looking computer monitor with apple logo center bottom, and on screen at top centered in light blue font letters is printed \"APPLE II\"\n\nDecomposed form: \"An older looking model computer components.\"|\"Computer disk drive boxes stacked.\"|\"A keyboard unit.\"|\"A modern computer monitor with apple logo at the center bottom.\"|\"On the screen of a modern computer monitor, at the top center, light blue font letters print the words APPLE II.\"\n\nExample 4:\nOriginal: A motel-like room features all of its furniture in rows, including a sofa, double bed, stuffed chair, desk chair, desk, bureau and television, all of which, like the one piece of artwork, carpet, linen and window drapes, are all in neutrals.\n\nDecomposed form: \"A motel room.\"|\"A neutral colored sofa.\"|\"A neutral colored double bed.\"|\"A neutral colored stuffed chair.\"|\"A desk chair.\"|\"A desk.\"|\"A bureau.\"|\"A television.\"|\"One piece of artwork.\"|\"Neutral colored linen.\"|\"Neutral colored window drapes.\"\n\nExample 5:\nOriginal: A room shows a twin bed, crib, double bed, and a round bed, all featuring checks and pillows, either white, or checked, high windows, a brick wall, a piece of art against a white wall, and a separate vertical surface with framed items on it.\n\nDecomposed form: \"A room.\"|\"A twin bed featuring checks and pillows, either white, or checked.\"|\"A crib.\"|\"A double bed featuring checks and pillows, either white, or checked.\"|\"A round bed featuring checks and pillows, either white, or checked.\"|\"High windows.\"|\"A brick wall.\"|\"A piece of art against a white wall.\"|\"A vertical surface with framed items on it.\"\n\nExample 6:\nOriginal: A street view shows a building entrance and traffic, including a bus, in the background, and to the foreground, a black iron fence, and a bench with an older, seated person, looking at a flock of pigeons scattered on the ground and bench.\n\nDecomposed form: \"A street view shows a building entrance.\"|\"A street view shows traffic, including a bus in the background.\"|\"In the foreground is a black iron fence.\"|\"In the foreground is a bench with an older person sitting on it\"|\"A flock of pigeons scattered on the ground and bench.\"\n\nExample 7:\nOriginal: Off in the distance, a few people dot the stands and the fenced in area bordering a ball field, while an umpire and three uniformed baseball players, including a batter, catcher and outfielder, strike playing poses in the foreground.\n\nDecomposed form: \"A few people dot the stands bordering a ball field.\"|\"A few people dot the fenced-in area bordering a ball field.\"|\"A baseball umpire.\"|\"A baseball batter.\"|\"A baseball catcher.\"|\"An outfielder.\"\n\nExample 8:\nOriginal: A counter top with a plate with a fork and few scraps of food and a teddy bear lying on side with arm outstretched on plate near fork, with another plate with an apple and two bowls with produce, a canister and some metal objects.\n\nDecomposed form: \"A counter top.\"|\"A plate with a fork and few scraps of food.\"|\"A teddy bear lying with arm outstretched on plate near fork.\"|\"A plate with an apple.\"|\"Bowls with apples.\"|\"A canister.\"|\"Metal objects.\"\n\nExample 9:\nOriginal: Indirect lighting in a darkened room plays across a table covered with the remains of a meal, thereby giving everything, including the table, a basket, plates with uneaten food, utensils and glasses with liquid, a reddish patina.\n\nDecomposed form: \"Indirect lighting in a darkened room shines across a table covered with the remains of a meal.\"|\"The table has a reddish patina.\"|\"The basket has a reddish patina.\"|\"The plates with uneaten food have a reddish patina.\"|\"Utensils have a reddish patina.\"|\"Glasses with liquid have a reddish patina.\"\n\nExample 10:\nOriginal: the long view of a parking lot shows a row of parked vehicles, including a bus, which appears to be the destination of a number of similarly dressed women with luggage that are moving towards it from the immediate foreground.\n\nDecomposed form: \"A parking lot shows a row of parked vehicles.\"|\"A bus.\"|\"Similarly dressed women with luggages.\"\n\nExample 11:\nOriginal: Corner view of a table with plaid cloth, red and white checkerboard napkins, a pizza, a container of green leafy vegetable, and some bottles, with the hips of a person wearing an apron standing beside table in green grass.\n\nDecomposed form: \"Corner view of a table with plaid cloth.\"|\"Red and white checkerboard napkins.\"|\"A pizza.\"|\"A container of green leafy vegetables.\"|\"Some bottles.\"|\"The hips of a person wearing an apron standing beside a table in green grass.\"\n\nExample 12:\nOriginal: A room with natural, unfinished, pale wood walls and built in shelves, also has curtained sliding doors that open onto a patio with a lounge chair, a big double bed with a blue blanket, and various framed photos and books.\n\nDecomposed form: \"A room.\"|\"Natural, unfinished, pale wood walls.\"|\"Built-in shelves.\"|\"Curtained sliding doors.\"|\"Sliding doors that open onto a patio with a lounge chair.\"|\"A big double bed with a blue blanket.\"|\"Framed photos.\"|\"Books.\"\n\nExample 13:\nOriginal: A minimally furnished room shows small window pairs with blinds and no curtains, an unmade double bed with a heavy, taupe duvet, and a white border near the ceiling with colorful birds trailing in both directions.\n\nDecomposed form: \"A minimally furnished room.\"|\"Small window pairs with blinds and no curtains.\"|\"An unmade double bed with a heavy, taupe duvet.\"|\"A white border near the ceiling with colorful birds trailing in both directions.\"\n\nExample 14:\nOriginal: A blurry night-time street scene shows a higher and lower roadway with overpasses and lots of traffic going the same way, while to the left an area of red glare with an arrow and number, suggests a caution area.\n\nDecomposed form: \"A blurry night-time street scene shows a higher and lower roadway with overpasses and lots of traffic going the same way.\"|\"An area of red glare with an arrow and number, suggests a caution area.\"\n\nExample 15:\nOriginal: Several people are standing, or sitting, in the vicinity of a scoreboard, to the front of which lies a tennis court, featuring a player with his knees bent, who is also grasping a racquet with both hands.\n\nDecomposed form: \"Several people are standing, or sitting, in the vicinity of a scoreboard.\"|\"In front of the scoreboard lies a tennis court.\"|\"A tennis player with his knees bent, who is grasping a racquet with both hands.\"\n\nExample 16:\nOriginal: Black and white of a blacksmith interior with a horse, a man standing with arm out to it, and a man ducking under it's head and holding a long metal file in both hands, with horses on poles on the wall.\n\nDecomposed form: \"Black and white photo of a blacksmith interior with a horse.\"|\"Black and white photo of a man standing with arm out to a horse.\"|\"Black and white photo of a man ducking under a horses head and holding a long metal file in both hands.\"|\"Black and white photo with horse poles on the wall.\"\n\nExample 17:\nOriginal: A blurry living room scene suggests a wide screen TV with a game playing, bookshelves, a low cluttered table, a couch, and closest and clearest, a sports t-shirt with the number thirty-five on it.\n\nDecomposed form: \"A blurry living room scene.\"|\"A wide screen TV with a sports game playing.\"|\"Bookshelves.\"|\"A low cluttered table.\"|\"A couch.\"|\"A sports t-shirt with the number 35 on it.\"\n\nExample 18:\nOriginal: In the first picture a boy is flipping his skateboard, in the second picture is skating and lifting the board slightly off the ground, and in the third he is jumping into the air onto the skateboard.\n\nDecomposed form: \"A boy is flipping his skateboard.\"|\"A boy is skating and lifting the skateboard slightly off the ground.\"|\"A boy is jumping into the air onto the skateboard.\"\n\nExample 19:\nOriginal: A table with a person holding a sandwich in hand over a plate with a knife on it, and another plate with a knife, napkin, and french bread sandwich, along with a bottle and a glass with beverages.\n\nDecomposed form: \"A table with a person holding a sandwich in hand.\"|\"A sandwich in hand over a plate with a knife on it.\"|\"A plate with a knife, napkin, and french bread sandwich.\"|\"A bottle.\"|\"A glass with beverages.\"\n\nExample 20:\nOriginal: An impressive array of motorcycles recedes into the distance between a driveway and a building fronted with cars and a grassy area, as lots of people mill about the driveway, around the bikes.\n\nDecomposed form: \"An array of motorcycles between a driveway and a building.\"|\"A building fronted with cars and a grassy area.\"|\"Lots of people mill about the driveway, around the bikes.\"\n\nGiven these examples of decomposing original sentences into their decomposed forms, decompose the given sentence:\n{sentence}\nPlease response in the same format, outputing only the decomposed sentences."}
        ]
    }
    for attempt in range(retries):
        try:
            async with session.post(
                url='https://api.openai.com/v1/chat/completions',
                headers={"Content-Type": "application/json", "Authorization": f"Bearer {OPENAI_SECRET_KEY}"},
                json=payload,
                ssl=ssl.create_default_context(cafile=certifi.where())
            ) as response:
                response_json = await response.json()
            if "error" in response_json:
                print(f"OpenAI request failed with error {response_json['error']}")
                if response_json['error']['code'] == 'rate_limit_exceeded':
                    retry_after = 10
                    await asyncio.sleep(retry_after)  # Wait for the retry-after duration before retrying
                continue
            result = response_json['choices'][0]['message']['content']
            print(result)
            return result
        except Exception as e:
            print(f"Request failed: {e}")
    print("errors")
    return ""

# Asynchronous function to process multiple sentences in parallel for decomposition
async def process_sentences_for_decomposition(sentences):
    async with aiohttp.ClientSession() as session:
        results = []
        for i in range(0, len(sentences), BATCH_SIZE):
            batch = sentences[i:i+BATCH_SIZE]
            tasks = [decompose_sentence_async(session, sentence) for sentence in batch]
            batch_results = await asyncio.gather(*tasks)
            results.extend(batch_results)
            if i + BATCH_SIZE < len(sentences):
                await asyncio.sleep(10)  # Wait for 10 seconds between batches to respect rate limits
    return results

def apply_decomposition_to_dataframe(df, sentence_col):
    # Create a copy of the DataFrame to avoid SettingWithCopyWarning
    df_copy = df.copy()
    sentences = df_copy[sentence_col].tolist()
    results = asyncio.run(process_sentences_for_decomposition(sentences))
    df_copy['caption_triples_ls'] = results
    return df_copy

if __name__ == "__main__":
    begin = time.time()
    df = df_mscoco[0:10].copy()  # Assuming tgt_df is already defined
    # Apply function to DataFrame
    df = apply_decomposition_to_dataframe(df, 'caption_mscoco')
    # Filter rows that do not start and end with "
    filtered_df = df[~(df['caption_triples_ls'].str.startswith('"') & df['caption_triples_ls'].str.endswith('"'))].copy()
    # Apply the lambda function to those rows only
    filtered_df['caption_triples_ls'] = filtered_df['caption_triples_ls'].apply(lambda x: '"' + x.split('"', 1)[-1].rsplit('"', 1)[0] + '"')
    # Combine the processed rows back with the original DataFrame
    result_df = pd.concat([df[(df['caption_triples_ls'].str.startswith('"') & df['caption_triples_ls'].str.endswith('"'))], filtered_df]).sort_index()
    end = time.time()
    total_time = end - begin
    print(f"Operation took a total of {total_time} seconds.")
    '''
    pickle_file_path = '/content/drive/MyDrive/mscoco_500decomposed.pkl'
    with open(pickle_file_path, 'wb') as f:
        pickle.dump(result_df, f)
    '''

"A parking lot shows a row of parked vehicles."|"A bus."|"Similarly dressed women with luggage moving towards a bus from the immediate foreground."
"A few people dot the stands bordering a ball field."|"A few people dot the fenced-in area bordering a ball field."|"A baseball umpire."|"A baseball batter."|"A baseball catcher."|"An outfielder."
"In a black night, a blurry shot shows a tall fence that angles, in a kind of Z shape."|"Around appears to be a parking area with a large building fronting it."|"A kid with a red helmet takes a fierce swing in the batting area, inside the fence."
"Indirect lighting in a darkened room shines across a table covered with the remains of a meal."|"The table has a reddish patina."|"The basket has a reddish patina."|"The plates with uneaten food have a reddish patina."|"Utensils have a reddish patina."|"Glasses with liquid have a reddish patina."
"An exit sign."|"Chairs."|"Recording equipment."|"A man with one arm aloft appears to be demonstrating someth

In [8]:
intermediate_df = result_df

In [9]:
intermediate_df

,id,image,caption_sharegpt4v,caption_mscoco,embedding,original_mscoco_caption_length,caption_triples_ls
0,553992,000000553992.jpg,"In the image, a man and a woman are standing i...",An institutional looking room holds an exit si...,"[0.25389287, 0.2295701, -0.108173296, -0.07395...",249,"""An exit sign.""|""Chairs.""|""Recording equipment..."
1,39270,000000039270.jpg,"In the image, there is a man who is in the mid...","Black night, a blurry shot shows a tall fence ...","[0.044150963, 0.17922473, -0.14461896, -0.1534...",246,"""In a black night, a blurry shot shows a tall ..."
2,277161,000000277161.jpg,"In the center of the image, a vintage Apple II...",An older looking model computer components; tw...,"[-0.41152698, -0.29223415, 0.025055759, -0.164...",244,"""An older looking model computer components.""|..."
3,380070,000000380070.jpg,"The image captures a cozy hotel room, bathed i...",A motel-like room features all of its furnitu...,"[0.23270163, 0.07637343, 0.36209187, -0.012398...",243,"""A motel-like room.""|""All of its furniture in ..."
4,357272,000000357272.jpg,The image captures a room that exudes a vintag...,"A room shows a twin bed, crib, double bed, and...","[0.45787933, 0.13048139, 0.11703592, 0.0351443...",242,"""A room.""|""A twin bed featuring checks and pil..."
5,480770,000000480770.jpg,"In the heart of a bustling city, an elderly wo...",A street view shows a building entrance and tr...,"[0.20354465, -0.0718597, -0.27712327, -0.49626...",241,"""A street view shows a building entrance.""|""A ..."
6,515241,000000515241.jpg,The image captures a dynamic moment in a baseb...,"Off in the distance, a few people dot the stan...","[-0.029054046, 0.23667967, 0.111569144, 0.0622...",235,"""A few people dot the stands bordering a ball ..."
7,560604,000000560604.jpg,"In the center of the image, a black stuffed an...",A counter top with a plate with a fork and few...,"[-0.29823345, 0.32871175, -0.07593347, -0.1097...",230,"""A counter top.""|""A plate with a fork and a fe..."
8,158005,000000158005.jpg,"In the dimly lit ambiance of the room, a dinne...",Indirect lighting in a darkened room plays acr...,"[-0.18686089, 0.35955262, 0.008806132, -0.3130...",229,"""Indirect lighting in a darkened room shines a..."
9,449712,000000449712.jpg,"In the image, a group of people are seen walki...",the long view of a parking lot shows a row of ...,"[0.19945762, -0.1157327, -0.03143066, 0.023830...",226,"""A parking lot shows a row of parked vehicles...."


Dependencies

In [11]:
import nest_asyncio
import asyncio
import aiohttp
import ssl
import certifi
import time
import pandas as pd
import pickle

nest_asyncio.apply()

# Settings
MODEL = "gpt-4o"
OPENAI_SECRET_KEY = "sk-proj-X...X"
MAX_RETRIES = 3  # Maximum number of retries
BATCH_SIZE = 25

class UnionFind:
    def __init__(self, size):
        self.parent = list(range(size))
        self.rank = [1] * size

    def find(self, p):
        if self.parent[p] != p:
            self.parent[p] = self.find(self.parent[p])  # Path compression
        return self.parent[p]

    def union(self, p, q):
        rootP = self.find(p)
        rootQ = self.find(q)
        if rootP != rootQ:
            # Union by rank
            if self.rank[rootP] > self.rank[rootQ]:
                self.parent[rootQ] = rootP
            elif self.rank[rootP] < self.rank[rootQ]:
                self.parent[rootP] = rootQ
            else:
                self.parent[rootQ] = rootP
                self.rank[rootP] += 1

# Asynchronous function to check dependency
async def check_dependency_async(session, sentence, segment1, segment2, retries=MAX_RETRIES):
    payload = {
        'model': MODEL,
        'messages': [
          {"role": "system", "content": "Example 1:\nOverall sentence: An institutional looking room holds an exit sign, chairs, recording equipment and a man with one arm aloft that appears to be demonstrating something with a device in the held-up hand for the benefit of a woman behind him that is watching closely.\nDecomposed sentence: \"An exit sign.\"|\"Chairs.\"|\"Recording equipment.\"|\"A man with one arm aloft appears to be demonstrating with a device in the held-up hand.\"|\"A woman behind a man is watching him closely.\"\nDependencies of decomposed sentences: 0 | 1 | 2 | 3,4\nExample 2:\nOverall sentence: Black night, a blurry shot shows a tall fence that angles, in a kind of Z shape, around what appears to be a parking area with a large building fronting it, as a kid with a red helmet takes a fierce swing in the batting area, inside the fence.\nDecomposed sentence: \"In a black night, a blurry shot shows a tall fence that angles, in a kind of Z shape.\"|\"Around appears to be a parking area with a large building fronting it.\"|\"A kid with a red helmet takes a fierce swing in the batting area, inside the fence.\"\nDependencies of decomposed sentences: 0,1,2\nExample 3:\nOverall sentence: An older looking model computer components; two disk drive boxes stacked, a keyboard unit, and a more modern looking computer monitor with apple logo center bottom, and on screen at top centered in light blue font letters is printed APPLE II\nDecomposed sentence: \"An older looking model computer components.\"|\"Computer disk drive boxes stacked.\"|\"A keyboard unit.\"|\"A modern computer monitor with apple logo at the center bottom.\"|\"On the screen of a modern computer monitor, at the top center, light blue font letters print the words APPLE II.\"\nDependencies of decomposed sentences: 0,1,2 | 3,4\nExample 4:\nOverall sentence: A motel-like room features all of its furniture in rows, including a sofa, double bed, stuffed chair, desk chair, desk, bureau and television, all of which, like the one piece of artwork, carpet, linen and window drapes, are all in neutrals.\nDecomposed sentence: \"A motel room.\"|\"A neutral colored sofa.\"|\"A neutral colored double bed.\"|\"A neutral colored stuffed chair.\"|\"A desk chair.\"|\"A desk.\"|\"A bureau.\"|\"A television.\"|\"One piece of artwork.\"|\"Neutral colored linen.\"|\"Neutral colored window drapes.\"\nDependencies of decomposed sentences: 0 | 1 | 2 | 3 | 4,5 | 6 | 7 | 8 | 9 | 10\nExample 5:\nOverall sentence: A room shows a twin bed, crib, double bed, and a round bed, all featuring checks and pillows, either white, or checked, high windows, a brick wall, a piece of art against a white wall, and a separate vertical surface with framed items on it.\nDecomposed sentence: \"A room.\"|\"A twin bed featuring checks and pillows, either white, or checked.\"|\"A crib.\"|\"A double bed featuring checks and pillows, either white, or checked.\"|\"A round bed featuring checks and pillows, either white, or checked.\"|\"High windows.\"|\"A brick wall.\"|\"A piece of art against a white wall.\"|\"A vertical surface with framed items on it.\"\nDependencies of decomposed sentences: 0 | 1| 2 | 3 | 4 | 5 | 6 | 7 | 8\nExample 6:\nOverall sentence: A street view shows a building entrance and traffic, including a bus, in the background, and to the foreground, a black iron fence, and a bench with an older, seated person, looking at a flock of pigeons scattered on the ground and bench.\nDecomposed sentence: \"A street view shows a building entrance.\"|\"A street view shows traffic, including a bus in the background.\"|\"In the foreground is a black iron fence.\"|\"In the foreground is a bench with an older person sitting on it\"|\"A flock of pigeons scattered on the ground and bench.\"\nDependencies of decomposed sentences: 0,1 | 2 | 3,4\nExample 7:\nOverall sentence: Off in the distance, a few people dot the stands and the fenced in area bordering a ball field, while an umpire and three uniformed baseball players, including a batter, catcher and outfielder, strike playing poses in the foreground.\nDecomposed form: \"A few people dot the stands bordering a ball field.\"|\"A few people dot the fenced-in area bordering a ball field.\"|\"A baseball umpire.\"|\"A baseball batter.\"|\"A baseball catcher.\"|\"An outfielder.\"\nDependencies of decomposed sentences: 0 | 1 | 2,3,4,5\nExample 8:\nOverall sentence: A counter top with a plate with a fork and few scraps of food and a teddy bear lying on side with arm outstretched on plate near fork, with another plate with an apple and two bowls with produce, a canister and some metal objects.\nDecomposed sentence: \"A counter top.\"|\"A plate with a fork and few scraps of food.\"|\"A teddy bear lying with arm outstretched on plate near fork.\"|\"A plate with an apple.\"|\"Bowls with apples.\"|\"A canister.\"|\"Metal objects.\"\nDependencies of decomposed sentences: 0 | 1,2 | 3 | 4 | 5 | 6\nExample 9:\nOverall sentence: Indirect lighting in a darkened room plays across a table covered with the remains of a meal, thereby giving everything, including the table, a basket, plates with uneaten food, utensils and glasses with liquid, a reddish patina.\nDecomposed sentence: \"Indirect lighting in a darkened room shines across a table covered with the remains of a meal.\"|\"The table has a reddish patina.\"|\"The basket has a reddish patina.\"|\"The plates with uneaten food have a reddish patina.\"|\"Utensils have a reddish patina.\"|\"Glasses with liquid have a reddish patina.\"\nDependencies of decomposed sentences: 0,1,2,3,4,5\nExample 10:\nOverall sentence: the long view of a parking lot shows a row of parked vehicles, including a bus, which appears to be the destination of a number of similarly dressed women with luggage that are moving towards it from the immediate foreground.\nDecomposed sentence: \"A parking lot shows a row of parked vehicles.\"|\"A bus.\"|\"Similarly dressed women with luggages.\"\nDependencies of decomposed sentences: 0,1 | 2\nExample 11:\nOverall sentence: Corner view of a table with plaid cloth, red and white checkerboard napkins, a pizza, a container of green leafy vegetable, and some bottles, with the hips of a person wearing an apron standing beside table in green grass.\nDecomposed sentence: \"Corner view of a table with plaid cloth.\"|\"Red and white checkerboard napkins.\"|\"A pizza.\"|\"A container of green leafy vegetables.\"|\"Some bottles.\"|\"The hips of a person wearing an apron standing beside a table in green grass.\"\nDependencies of decomposed sentences: 0,5| 1 | 2 | 3 | 4\nExample 12:\nOverall sentence: A room with natural, unfinished, pale wood walls and built in shelves, also has curtained sliding doors that open onto a patio with a lounge chair, a big double bed with a blue blanket, and various framed photos and books.\nDecomposed sentence: \"A room.\"|\"Natural, unfinished, pale wood walls.\"|\"Built-in shelves.\"|\"Curtained sliding doors.\"|\"Sliding doors that open onto a patio with a lounge chair.\"|\"A big double bed with a blue blanket.\"|\"Framed photos.\"|\"Books.\"\nDependencies of decomposed sentences: 0 | 1 | 2 | 3,4 | 5 | 6 | 7\nExample 13:\nOverall sentence: A minimally furnished room shows small window pairs with blinds and no curtains, an unmade double bed with a heavy, taupe duvet, and a white border near the ceiling with colorful birds trailing in both directions.\nDecomposed sentence: \"A minimally furnished room.\"|\"Small window pairs with blinds and no curtains.\"|\"An unmade double bed with a heavy, taupe duvet.\"|\"A white border near the ceiling with colorful birds trailing in both directions.\"\nDependencies of decomposed sentences: 0 | 1 | 2 | 3\nExample 14:\nOverall sentence: A blurry night-time street scene shows a higher and lower roadway with overpasses and lots of traffic going the same way, while to the left an area of red glare with an arrow and number, suggests a caution area.\nDecomposed sentence: \"A blurry night-time street scene shows a higher and lower roadway with overpasses and lots of traffic going the same way.\"|\"An area of red glare with an arrow and number, suggests a caution area.\"\nDependencies of decomposed sentences: 0,1\nExample 15:\nOverall sentence: Several people are standing, or sitting, in the vicinity of a scoreboard, to the front of which lies a tennis court, featuring a player with his knees bent, who is also grasping a racquet with both hands,\nDecomposed sentence: \"Several people are standing, or sitting, in the vicinity of a scoreboard.\"|\"In front of the scoreboard lies a tennis court.\"|\"A tennis player with his knees bent, who is grasping a racquet with both hands.\"\nDependencies of decomposed sentences: 0,1 | 2\nExample 16:\nOverall sentence: Black and white of a blacksmith interior with a horse, a man standing with arm out to it, and a man ducking under it's head and holding a long metal file in both hands, with horses on poles on the wall.\nDecomposed sentence: \"Black and white photo of a blacksmith interior with a horse.\"|\"Black and white photo of a man standing with arm out to a horse.\"|\"Black and white photo of a man ducking under a horses head and holding a long metal file in both hands.\"|\"Black and white photo with horse poles on the wall.\"\nDependencies of decomposed sentences: 0,1,2 | 3\nExample 17:\nOverall sentence: A blurry living room scene suggests a wide screen T,V. with a game playing, bookshelves, a low cluttered table, a couch, and closest and clearest, a sports t-shirt with the number thirty-five on it.\nDecomposed sentence: \"A blurry living room scene.\"|\"A wide screen TV with a sports game playing.\"|\"Bookshelves.\"|\"A low cluttered table.\"|\"A couch.\"|\"A sports t-shirt with the number 35 on it.\"\nDependencies of decomposed sentences: 0 | 1 | 2 | 3 | 4 | 5\nExample 18:\nOverall sentence: In the first picture a boy is flipping his skateboard, in the second picture is skating and lifting the board slightly off the ground, and in the third he is jumping into the air onto the skateboard.\nDecomposed sentence: \"A boy is flipping his skateboard.\"|\"A boy is skating and lifting the skateboard slightly off the ground.\"|\"A boy is jumping into the air onto the skateboard.\"\nDependencies of decomposed sentences: 0 | 1 | 2\nExample 19:\nOverall sentence: A table with a person holding a sandwich in hand over a plate with a knife on it, and another plate with a knife, napkin, and french bread sandwich, along with a bottle and a glass with beverages.\nDecomposed sentence: \"A table with a person holding a sandwich in hand.\"|\"A sandwich in hand over a plate with a knife on it.\"|\"A plate with a knife, napkin, and french bread sandwich.\"|\"A bottle.\"|\"A glass with beverages.\"\nDependencies of decomposed sentences: 0,1 | 2,3,4\nExample 20:\nOverall sentence: An impressive array of motorcycles recedes into the distance between a driveway and a building fronted with cars and a grassy area, as lots of people mill about the driveway, around the bikes.\nDecomposed sentence: \"An array of motorcycles between a driveway and a building.\"|\"A building fronted with cars and a grassy area.\"|\"Lots of people mill about the driveway, around the bikes.\"\nDependencies of decomposed sentences: 0,1,2\nLearning from how the provided 20 examples of overall sentences have their decomposed phrases, each represented by their respective index, starting from 0, are decomposed, then the decomposed phrases are grouped together by commas to indicate phrases are dependent, while bars separate independent phrases or phrase groups, output whether two decomposed phrases are dependent given the overall sentence. You may answer in only one digit, 1 meaning yes, 0 meaning no."},
          {"role": "user", "content": f"Overall sentence:{sentence}\n And two decomposed phrases: {segment1}|{segment2}\nPlease output 1 or 0, 1 meaning they are dependent phrases, 0 meaning they are independent phraes."}
        ],
        'max_tokens': 1,
        'n': 1,
        'temperature': 0
    }
    for attempt in range(retries):
        try:
            async with session.post(
                url='https://api.openai.com/v1/chat/completions',
                headers={"Content-Type": "application/json", "Authorization": f"Bearer {OPENAI_SECRET_KEY}"},
                json=payload,
                ssl=ssl.create_default_context(cafile=certifi.where())
            ) as response:
                response_json = await response.json()
            if "error" in response_json:
                print(f"OpenAI request failed with error {response_json['error']}")
                if response_json['error']['code'] == 'rate_limit_exceeded':
                    retry_after = 10
                    await asyncio.sleep(retry_after)  # Wait for the retry-after duration before retrying
                continue
            result = response_json['choices'][0]['message']['content']
            if result in {'1', '0'}:
                print(result)
                return int(result)
            else:
                print("errors")
                return 0
        except Exception as e:
            print(f"Request failed: {e}")
    print("errors")
    return 0

# Asynchronous function to group dependent segments for a single sentence
async def group_dependent_segments_async(session, sentence, segments):
    start_time = time.time()  # Start time
    segments_list = segments.split('|')
    n = len(segments_list)

    # Initialize Union-Find
    uf = UnionFind(n)

    # Perform pairwise comparisons sequentially
    for i in range(n):
        for j in range(i + 1, n):
            if uf.find(i) != uf.find(j):  # Check if i and j are already in the same group
                result = await check_dependency_async(session, sentence, segments_list[i], segments_list[j])
                if result == 1:
                    uf.union(i, j)
                await asyncio.sleep(2)  # Small delay to avoid hitting limits too quickly
            else:
              print('already in same group')

    # Group segments based on union-find results
    groups = {}
    for i in range(n):
        root = uf.find(i)
        if root not in groups:
            groups[root] = []
        groups[root].append(i)

    # Convert indices back to segments
    grouped_indices = [",".join(map(str, sorted(group))) for group in groups.values()]
    end_time = time.time()  # End time
    elapsed_time = end_time - start_time  # Calculate elapsed time
    return " | ".join(grouped_indices), elapsed_time

# Asynchronous function to process multiple sentences in parallel
async def process_sentences_in_bulk(sentences_and_segments):
    async with aiohttp.ClientSession() as session:
        results = []
        for i in range(0, len(sentences_and_segments), BATCH_SIZE):
            batch = sentences_and_segments[i:i+BATCH_SIZE]
            tasks = [group_dependent_segments_async(session, sentence, segments) for sentence, segments in batch]
            batch_results = await asyncio.gather(*tasks)
            results.extend(batch_results)
            if i + BATCH_SIZE < len(sentences_and_segments):
                await asyncio.sleep(10)  # Wait for 60 seconds between batches to respect rate limits
    return results

def apply_to_dataframe(df, sentence_col, segments_col):
    # Create a copy of the DataFrame to avoid SettingWithCopyWarning
    df_copy = df.copy()
    sentences_and_segments = list(zip(df_copy[sentence_col], df_copy[segments_col]))
    results = asyncio.run(process_sentences_in_bulk(sentences_and_segments))
    df_copy['groups'] = [result[0] for result in results]
    df_copy['elapsed_time'] = [result[1] for result in results]
    return df_copy

if __name__ == "__main__":
    begin = time.time()
    df = intermediate_df.copy() #can change according to user
    # Filter rows that do not start and end with "
    filtered_df = df[~(df['caption_triples_ls'].str.startswith('"') & df['caption_triples_ls'].str.endswith('"'))].copy()
    # Apply the lambda function to those rows only
    filtered_df['caption_triples_ls'] = filtered_df['caption_triples_ls'].apply(lambda x: '"' + x.split('"', 1)[-1].rsplit('"', 1)[0] + '"')
    # Combine the processed rows back with the original DataFrame
    result_df = pd.concat([df[(df['caption_triples_ls'].str.startswith('"') & df['caption_triples_ls'].str.endswith('"'))], filtered_df]).sort_index()
    # Apply function to DataFrame
    final_df = apply_to_dataframe(result_df, 'caption_mscoco', 'caption_triples_ls')
    end = time.time()
    total_time = end - begin
    print(f"Operation took a total of {total_time} seconds.")
    '''
    pickle_file_path = '/content/drive/MyDrive/mscoco_500decomposed_dependencies.pkl'
    with open(pickle_file_path, 'wb') as f:
      pickle.dump(final_df, f)
    '''

1
0
0
1
1
0
1
1
1
1
0
0
0
0
0
1
0
1
0
1
already in same group
1
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
1
0
0
1
0
0
0
0
0
1
0
1
0
0
0
already in same group
0
0
1
0
0
1
0
0
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
0
1
0
0
0
0
0
0
1
0
0
0
0
0
1
1
1
0
1
0
1
1
0
0
0
1
0
0
already in same group
already in same group
already in same group
0
0
already in same group
already in same group
already in same group
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
1
0
0
already in same group
already in same group
1
0
0
0
already in same group
0
0
0
0
0
0
0
1
already in same group
0
1
already in same group
0
0
0
0
0
0
0
0
0
0
already in same group
already in same group
already in same group
0
0
0
0
1
0
0
0
0
already in same group
already in same group
already in same group
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
already in same group
already in same group
already in same group


In [12]:
final_df

,id,image,caption_sharegpt4v,caption_mscoco,embedding,original_mscoco_caption_length,caption_triples_ls,groups,elapsed_time
0,553992,000000553992.jpg,"In the image, a man and a woman are standing i...",An institutional looking room holds an exit si...,"[0.25389287, 0.2295701, -0.108173296, -0.07395...",249,"""An exit sign.""|""Chairs.""|""Recording equipment...","0 | 1 | 2 | 3,4",30.251717
1,39270,000000039270.jpg,"In the image, there is a man who is in the mid...","Black night, a blurry shot shows a tall fence ...","[0.044150963, 0.17922473, -0.14461896, -0.1534...",246,"""In a black night, a blurry shot shows a tall ...","0,1,2",9.223561
2,277161,000000277161.jpg,"In the center of the image, a vintage Apple II...",An older looking model computer components; tw...,"[-0.41152698, -0.29223415, 0.025055759, -0.164...",244,"""An older looking model computer components.""|...","0,1,2,3,4",26.510803
3,380070,000000380070.jpg,"The image captures a cozy hotel room, bathed i...",A motel-like room features all of its furnitu...,"[0.23270163, 0.07637343, 0.36209187, -0.012398...",243,"""A motel-like room.""|""All of its furniture in ...","0,1,2,3,4,9,10,11,12 | 5 | 6 | 7,8",169.059217
4,357272,000000357272.jpg,The image captures a room that exudes a vintag...,"A room shows a twin bed, crib, double bed, and...","[0.45787933, 0.13048139, 0.11703592, 0.0351443...",242,"""A room.""|""A twin bed featuring checks and pil...","0 | 1,3,4 | 2 | 5 | 6,7 | 8",102.877314
5,480770,000000480770.jpg,"In the heart of a bustling city, an elderly wo...",A street view shows a building entrance and tr...,"[0.20354465, -0.0718597, -0.27712327, -0.49626...",241,"""A street view shows a building entrance.""|""A ...","0,1 | 2,3,4",29.058734
6,515241,000000515241.jpg,The image captures a dynamic moment in a baseb...,"Off in the distance, a few people dot the stan...","[-0.029054046, 0.23667967, 0.111569144, 0.0622...",235,"""A few people dot the stands bordering a ball ...","0,1 | 2,3,4,5",34.213959
7,560604,000000560604.jpg,"In the center of the image, a black stuffed an...",A counter top with a plate with a fork and few...,"[-0.29823345, 0.32871175, -0.07593347, -0.1097...",230,"""A counter top.""|""A plate with a fork and a fe...","0,1,2 | 3,4,5,6",59.806925
8,158005,000000158005.jpg,"In the dimly lit ambiance of the room, a dinne...",Indirect lighting in a darkened room plays acr...,"[-0.18686089, 0.35955262, 0.008806132, -0.3130...",229,"""Indirect lighting in a darkened room shines a...","0,1,2,3,4,5",20.641703
9,449712,000000449712.jpg,"In the image, a group of people are seen walki...",the long view of a parking lot shows a row of ...,"[0.19945762, -0.1157327, -0.03143066, 0.023830...",226,"""A parking lot shows a row of parked vehicles....","0,1,2",6.220107


V2

In [13]:
import nest_asyncio
import asyncio
import aiohttp
import ssl
import certifi
import time
import pandas as pd
import pickle

nest_asyncio.apply()

# Settings
MODEL = "gpt-4o"
OPENAI_SECRET_KEY = "sk-proj-X...X"
MAX_RETRIES = 3  # Maximum number of retries
BATCH_SIZE = 35

class UnionFind:
    def __init__(self, size):
        self.parent = list(range(size))
        self.rank = [1] * size

    def find(self, p):
        if self.parent[p] != p:
            self.parent[p] = self.find(self.parent[p])  # Path compression
        return self.parent[p]

    def union(self, p, q):
        rootP = self.find(p)
        rootQ = self.find(q)
        if rootP != rootQ:
            # Union by rank
            if self.rank[rootP] > self.rank[rootQ]:
                self.parent[rootQ] = rootP
            elif self.rank[rootP] < self.rank[rootQ]:
                self.parent[rootP] = rootQ
            else:
                self.parent[rootQ] = rootP
                self.rank[rootP] += 1

# Asynchronous function to check dependency
async def check_dependency_async(session, sentence, segment1, segment2, retries=MAX_RETRIES):
    payload = {
        'model': MODEL,
        'messages': [
          {"role": "system", "content": "Example 1:\nOverall sentence: An institutional looking room holds an exit sign, chairs, recording equipment and a man with one arm aloft that appears to be demonstrating something with a device in the held-up hand for the benefit of a woman behind him that is watching closely.\nDecomposed sentence: [\"An exit sign.\"]|[\"Chairs.\"]|[\"Recording equipment.\"]|[\"A man with one arm aloft appears to be demonstrating with a device in the held-up hand.\",\"A woman behind a man is watching him closely.\"]\nExample 2:\nOverall sentence: Black night, a blurry shot shows a tall fence that angles, in a kind of Z shape, around what appears to be a parking area with a large building fronting it, as a kid with a red helmet takes a fierce swing in the batting area, inside the fence.\nDecomposed sentence: [\"In a black night, a blurry shot shows a tall fence that angles, in a kind of Z shape.\",\"Around appears to be a parking area with a large building fronting it.\",\"A kid with a red helmet takes a fierce swing in the batting area, inside the fence.\"]\nExample 3:\nOverall sentence: An older looking model computer components; two disk drive boxes stacked, a keyboard unit, and a more modern looking computer monitor with apple logo center bottom, and on screen at top centered in light blue font letters is printed APPLE II\nDecomposed sentence: [\"An older looking model computer components.\",\"Computer disk drive boxes stacked.\",\"A keyboard unit.\"]|[\"A modern computer monitor with apple logo at the center bottom.\",\"On the screen of a modern computer monitor, at the top center, light blue font letters print the words APPLE II.\"]\nExample 4:\nOverall sentence: A motel-like room features all of its furniture in rows, including a sofa, double bed, stuffed chair, desk chair, desk, bureau and television, all of which, like the one piece of artwork, carpet, linen and window drapes, are all in neutrals.\nDecomposed sentence: [\"A motel room.\"]|[\"A neutral colored sofa.\"]|[\"A neutral colored double bed.\"]|[\"A neutral colored stuffed chair.\"]|[\"A desk chair.\",\"A desk.\"]|[\"A bureau.\"]|[\"A television.\"]|[\"One piece of artwork.\"]|[\"Neutral colored linen.\"]|[\"Neutral colored window drapes.\"]\nExample 5:\nOverall sentence: A room shows a twin bed, crib, double bed, and a round bed, all featuring checks and pillows, either white, or checked, high windows, a brick wall, a piece of art against a white wall, and a separate vertical surface with framed items on it.\nDecomposed sentence: [\"A room.\"]|[\"A twin bed featuring checks and pillows, either white, or checked.\"]|[\"A crib.\"]|[\"A double bed featuring checks and pillows, either white, or checked.\"]|[\"A round bed featuring checks and pillows, either white, or checked.\"]|[\"High windows.\"]|[\"A brick wall.\"]|[\"A piece of art against a white wall.\"]|[\"A vertical surface with framed items on it.\"]\nExample 6:\nOverall sentence: A street view shows a building entrance and traffic, including a bus, in the background, and to the foreground, a black iron fence, and a bench with an older, seated person, looking at a flock of pigeons scattered on the ground and bench.\nDecomposed sentence: [\"A street view shows a building entrance.\",\"A street view shows traffic, including a bus in the background.\"]|[\"In the foreground is a black iron fence.\"]|[\"In the foreground is a bench with an older person sitting on it\",\"A flock of pigeons scattered on the ground and bench.\"]\nExample 7:\nOverall sentence: Off in the distance, a few people dot the stands and the fenced in area bordering a ball field, while an umpire and three uniformed baseball players, including a batter, catcher and outfielder, strike playing poses in the foreground.\nDecomposed form: [\"A few people dot the stands bordering a ball field.\"]|[\"A few people dot the fenced-in area bordering a ball field.\"]|[\"A baseball umpire.\",\"A baseball batter.\",\"A baseball catcher.\",\"An outfielder.\"]\nExample 8:\nOverall sentence: A counter top with a plate with a fork and few scraps of food and a teddy bear lying on side with arm outstretched on plate near fork, with another plate with an apple and two bowls with produce, a canister and some metal objects.\nDecomposed sentence: [\"A counter top.\"]|[\"A plate with a fork and few scraps of food.\",\"A teddy bear lying with arm outstretched on plate near fork.\"]|[\"A plate with an apple.\"]|[\"Bowls with apples.\"]|[\"A canister.\"]|[\"Metal objects.\"]\nExample 9:\nOverall sentence: Indirect lighting in a darkened room plays across a table covered with the remains of a meal, thereby giving everything, including the table, a basket, plates with uneaten food, utensils and glasses with liquid, a reddish patina.\nDecomposed sentence: [\"Indirect lighting in a darkened room shines across a table covered with the remains of a meal.\",\"The table has a reddish patina.\",\"The basket has a reddish patina.\",\"The plates with uneaten food have a reddish patina.\",\"Utensils have a reddish patina.\",\"Glasses with liquid have a reddish patina.\"]\nExample 10:\nOverall sentence: the long view of a parking lot shows a row of parked vehicles, including a bus, which appears to be the destination of a number of similarly dressed women with luggage that are moving towards it from the immediate foreground.\nDecomposed sentence: [\"A parking lot shows a row of parked vehicles.\",\"A bus.\"]|[\"Similarly dressed women with luggages.\"]\nExample 11:\nOverall sentence: Corner view of a table with plaid cloth, red and white checkerboard napkins, a pizza, a container of green leafy vegetable, and some bottles, with the hips of a person wearing an apron standing beside table in green grass.\nDecomposed sentence: [\"Corner view of a table with plaid cloth.\",\"The hips of a person wearing an apron standing beside a table in green grass.\"]|[\"Red and white checkerboard napkins.\"]|[\"A pizza.\"]|[\"A container of green leafy vegetables.\"]|[\"Some bottles.\"]\nExample 12:\nOverall sentence: A room with natural, unfinished, pale wood walls and built in shelves, also has curtained sliding doors that open onto a patio with a lounge chair, a big double bed with a blue blanket, and various framed photos and books.\nDecomposed sentence: [\"A room.\"]|[\"Natural, unfinished, pale wood walls.\"]|[\"Built-in shelves.\"]|[\"Curtained sliding doors.\",\"Sliding doors that open onto a patio with a lounge chair.\"]|[\"A big double bed with a blue blanket.\"]|[\"Framed photos.\"]|[\"Books.\"]\nExample 13:\nOverall sentence: A minimally furnished room shows small window pairs with blinds and no curtains, an unmade double bed with a heavy, taupe duvet, and a white border near the ceiling with colorful birds trailing in both directions.\nDecomposed sentence: [\"A minimally furnished room.\"]|[\"Small window pairs with blinds and no curtains.\"]|[\"An unmade double bed with a heavy, taupe duvet.\"]|[\"A white border near the ceiling with colorful birds trailing in both directions.\"]\nExample 14:\nOverall sentence: A blurry night-time street scene shows a higher and lower roadway with overpasses and lots of traffic going the same way, while to the left an area of red glare with an arrow and number, suggests a caution area.\nDecomposed sentence: [\"A blurry night-time street scene shows a higher and lower roadway with overpasses and lots of traffic going the same way.\"]|[\"An area of red glare with an arrow and number, suggests a caution area.\"]\nExample 15:\nOverall sentence: Several people are standing, or sitting, in the vicinity of a scoreboard, to the front of which lies a tennis court, featuring a player with his knees bent, who is also grasping a racquet with both hands,\nDecomposed sentence: [\"Several people are standing, or sitting, in the vicinity of a scoreboard.\",\"In front of the scoreboard lies a tennis court.\"]|[\"A tennis player with his knees bent, who is grasping a racquet with both hands.\"]\nExample 16:\nOverall sentence: Black and white of a blacksmith interior with a horse, a man standing with arm out to it, and a man ducking under it's head and holding a long metal file in both hands, with horses on poles on the wall.\nDecomposed sentence: [\"Black and white photo of a blacksmith interior with a horse.\",\"Black and white photo of a man standing with arm out to a horse.\",\"Black and white photo of a man ducking under a horses head and holding a long metal file in both hands.\"]|[\"Black and white photo with horse poles on the wall.\"]\nExample 17:\nOverall sentence: A blurry living room scene suggests a wide screen T,V. with a game playing, bookshelves, a low cluttered table, a couch, and closest and clearest, a sports t-shirt with the number thirty-five on it.\nDecomposed sentence: [\"A blurry living room scene.\"]|[\"A wide screen TV with a sports game playing.\"]|[\"Bookshelves.\"]|[\"A low cluttered table.\"]|[\"A couch.\"]|[\"A sports t-shirt with the number 35 on it.\"]\nExample 18:\nOverall sentence: In the first picture a boy is flipping his skateboard, in the second picture is skating and lifting the board slightly off the ground, and in the third he is jumping into the air onto the skateboard.\nDecomposed sentence: [\"A boy is flipping his skateboard.\",\"A boy is skating and lifting the skateboard slightly off the ground.\",\"A boy is jumping into the air onto the skateboard.\"]\nExample 19:\nOverall sentence: A table with a person holding a sandwich in hand over a plate with a knife on it, and another plate with a knife, napkin, and french bread sandwich, along with a bottle and a glass with beverages.\nDecomposed sentence: [\"A table with a person holding a sandwich in hand.\",\"A sandwich in hand over a plate with a knife on it.\"]|[\"A plate with a knife, napkin, and french bread sandwich.\",\"A bottle.\",\"A glass with beverages.\"]\nExample 20:\nOverall sentence: An impressive array of motorcycles recedes into the distance between a driveway and a building fronted with cars and a grassy area, as lots of people mill about the driveway, around the bikes.\nDecomposed sentence: [\"An array of motorcycles between a driveway and a building.\",\"A building fronted with cars and a grassy area.\",\"Lots of people mill about the driveway, around the bikes.\"]\nIn the provided 20 examples, with overall sentence and their decomposed phrases, dependent phrases are grouped in the same set of brackets, while independent phrases are not grouped under the same set of brackets. Note that we define two sentences as dependent when the two phrases share an explicitly defined physical and material object between them in the overall sentence. Given an overall sentence and two decomposed phrases from user input, output whether the two decomposed phrases are dependent or independent. You may answer in only one digit, 1 meaning dependent, 0 meaning independent. Only output 1 if you are 100% certain. Do not consider any vague dependencies."},
          {"role": "user", "content": f"Overall sentence:{sentence}\n And two decomposed phrases: {segment1}, {segment2}\nPlease output 1 or 0, 1 meaning they are dependent phrases, 0 meaning they are independent phrases."}
        ],
        'max_tokens': 1,
        'n': 1,
        'temperature': 0
    }
    for attempt in range(retries):
        try:
            async with session.post(
                url='https://api.openai.com/v1/chat/completions',
                headers={"Content-Type": "application/json", "Authorization": f"Bearer {OPENAI_SECRET_KEY}"},
                json=payload,
                ssl=ssl.create_default_context(cafile=certifi.where())
            ) as response:
                response_json = await response.json()
            if "error" in response_json:
                print(f"OpenAI request failed with error {response_json['error']}")
                if response_json['error']['code'] == 'rate_limit_exceeded':
                    retry_after = 10
                    await asyncio.sleep(retry_after)  # Wait for the retry-after duration before retrying
                continue
            result = response_json['choices'][0]['message']['content']
            if result in {'1', '0'}:
                print(result)
                return int(result)
            else:
                print("errors")
                return 0
        except Exception as e:
            print(f"Request failed: {e}")
    print("errors")
    return 0

# Asynchronous function to group dependent segments for a single sentence
async def group_dependent_segments_async(session, sentence, segments):
    start_time = time.time()  # Start time
    segments_list = segments.split('|')
    n = len(segments_list)

    # Initialize Union-Find
    uf = UnionFind(n)

    # Perform pairwise comparisons sequentially
    for i in range(n):
        for j in range(i + 1, n):
            if uf.find(i) != uf.find(j):  # Check if i and j are already in the same group
                result = await check_dependency_async(session, sentence, segments_list[i], segments_list[j])
                if result == 1:
                    uf.union(i, j)
                await asyncio.sleep(2)  # Small delay to avoid hitting limits too quickly
            else:
              print('already in same group')

    # Group segments based on union-find results
    groups = {}
    for i in range(n):
        root = uf.find(i)
        if root not in groups:
            groups[root] = []
        groups[root].append(i)

    # Convert indices back to segments
    grouped_indices = [",".join(map(str, sorted(group))) for group in groups.values()]
    end_time = time.time()  # End time
    elapsed_time = end_time - start_time  # Calculate elapsed time
    return " | ".join(grouped_indices), elapsed_time

# Asynchronous function to process multiple sentences in parallel
async def process_sentences_in_bulk(sentences_and_segments):
    async with aiohttp.ClientSession() as session:
        results = []
        for i in range(0, len(sentences_and_segments), BATCH_SIZE):
            batch = sentences_and_segments[i:i+BATCH_SIZE]
            tasks = [group_dependent_segments_async(session, sentence, segments) for sentence, segments in batch]
            batch_results = await asyncio.gather(*tasks)
            results.extend(batch_results)
            if i + BATCH_SIZE < len(sentences_and_segments):
                await asyncio.sleep(10)  # Wait for 60 seconds between batches to respect rate limits
    return results

def apply_to_dataframe(df, sentence_col, segments_col):
    # Create a copy of the DataFrame to avoid SettingWithCopyWarning
    df_copy = df.copy()
    sentences_and_segments = list(zip(df_copy[sentence_col], df_copy[segments_col]))
    results = asyncio.run(process_sentences_in_bulk(sentences_and_segments))
    df_copy['groups'] = [result[0] for result in results]
    df_copy['elapsed_time'] = [result[1] for result in results]
    return df_copy

if __name__ == "__main__":
    begin = time.time()
    df = intermediate_df.copy()
    # Filter rows that do not start and end with "
    filtered_df = df[~(df['caption_triples_ls'].str.startswith('"') & df['caption_triples_ls'].str.endswith('"'))].copy()
    # Apply the lambda function to those rows only
    filtered_df['caption_triples_ls'] = filtered_df['caption_triples_ls'].apply(lambda x: '"' + x.split('"', 1)[-1].rsplit('"', 1)[0] + '"')
    # Combine the processed rows back with the original DataFrame
    result_df = pd.concat([df[(df['caption_triples_ls'].str.startswith('"') & df['caption_triples_ls'].str.endswith('"'))], filtered_df]).sort_index()
    # Apply function to DataFrame
    final_df = apply_to_dataframe(result_df, 'caption_mscoco', 'caption_triples_ls')
    end = time.time()
    total_time = end - begin
    print(f"Operation took a total of {total_time} seconds.")
    '''
    pickle_file_path = '/content/drive/MyDrive/mscoco_500decomposed_dependencies_v2.pkl'
    with open(pickle_file_path, 'wb') as f:
      pickle.dump(final_df, f)
    '''

1
1
1
1
1
1
0
1
1
1
0
1
1
0
1
0
1
0
0
1
already in same group
already in same group
0
1
1
0
0
0
0
0
1
1
0
0
1
0
0
0
0
1
1
0
0
0
0
0
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
0
1
0
0
0
0
0
already in same group
0
already in same group
0
0
0
0
0
0
0
1
1
0
0
0
0
1
0
1
0
0
0
0
1
1
1
0
1
1
1
0
1
1
1
0
1
0
already in same group
already in same group
already in same group
already in same group
already in same group
1
0
already in same group
already in same group
already in same group
0
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already 

In [14]:
final_df

,id,image,caption_sharegpt4v,caption_mscoco,embedding,original_mscoco_caption_length,caption_triples_ls,groups,elapsed_time
0,553992,000000553992.jpg,"In the image, a man and a woman are standing i...",An institutional looking room holds an exit si...,"[0.25389287, 0.2295701, -0.108173296, -0.07395...",249,"""An exit sign.""|""Chairs.""|""Recording equipment...","0 | 1 | 2 | 3,4",29.285209
1,39270,000000039270.jpg,"In the image, there is a man who is in the mid...","Black night, a blurry shot shows a tall fence ...","[0.044150963, 0.17922473, -0.14461896, -0.1534...",246,"""In a black night, a blurry shot shows a tall ...","0,1,2",6.904167
2,277161,000000277161.jpg,"In the center of the image, a vintage Apple II...",An older looking model computer components; tw...,"[-0.41152698, -0.29223415, 0.025055759, -0.164...",244,"""An older looking model computer components.""|...","0,1,3,4 | 2",24.884712
3,380070,000000380070.jpg,"The image captures a cozy hotel room, bathed i...",A motel-like room features all of its furnitu...,"[0.23270163, 0.07637343, 0.36209187, -0.012398...",243,"""A motel-like room.""|""All of its furniture in ...","0,1,2,3,4,5,6,7,8,9,10,11,12",36.476129
4,357272,000000357272.jpg,The image captures a room that exudes a vintag...,"A room shows a twin bed, crib, double bed, and...","[0.45787933, 0.13048139, 0.11703592, 0.0351443...",242,"""A room.""|""A twin bed featuring checks and pil...","0,1,3,4 | 2 | 5 | 6 | 7 | 8",96.079601
5,480770,000000480770.jpg,"In the heart of a bustling city, an elderly wo...",A street view shows a building entrance and tr...,"[0.20354465, -0.0718597, -0.27712327, -0.49626...",241,"""A street view shows a building entrance.""|""A ...","0,1 | 2,3,4",28.337717
6,515241,000000515241.jpg,The image captures a dynamic moment in a baseb...,"Off in the distance, a few people dot the stan...","[-0.029054046, 0.23667967, 0.111569144, 0.0622...",235,"""A few people dot the stands bordering a ball ...","0,1 | 2,3,4,5",35.038784
7,560604,000000560604.jpg,"In the center of the image, a black stuffed an...",A counter top with a plate with a fork and few...,"[-0.29823345, 0.32871175, -0.07593347, -0.1097...",230,"""A counter top.""|""A plate with a fork and a fe...","0,1,2 | 3 | 4 | 5,6",56.153500
8,158005,000000158005.jpg,"In the dimly lit ambiance of the room, a dinne...",Indirect lighting in a darkened room plays acr...,"[-0.18686089, 0.35955262, 0.008806132, -0.3130...",229,"""Indirect lighting in a darkened room shines a...","0,1,2,3,4,5",14.470415
9,449712,000000449712.jpg,"In the image, a group of people are seen walki...",the long view of a parking lot shows a row of ...,"[0.19945762, -0.1157327, -0.03143066, 0.023830...",226,"""A parking lot shows a row of parked vehicles....","0,1,2",6.133904
